In [4]:
!pip3 install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 MB 3.4 MB/s eta 0:00:0000:0100:01
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 3.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 3.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 3.7 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [26]:
import pandas as pd
import numpy as np
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
import torch
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf

In [6]:
def clean_text(text):
    # Menghilangkan karakter non-alfanumerik dan mengganti beberapa karakter
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Opsi tambahan: mengganti newline dengan spasi, dll.
    text = text.replace('\n', ' ').replace('\r', '')
    return text

In [7]:
# Inisialisasi Tokenizer mBERT
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

tokenizer_config.json: 100%|██████████| 29.0/29.0 [00:00<00:00, 11.0kB/s]
vocab.txt: 100%|██████████| 996k/996k [00:01<00:00, 603kB/s]
tokenizer.json: 100%|██████████| 1.96M/1.96M [00:01<00:00, 1.30MB/s]
config.json: 100%|██████████| 625/625 [00:00<00:00, 329kB/s]


In [8]:
def sliding_window_tokenize(text, max_length=512, stride=50):
    # Konversi teks ke token dan inisialisasi segmen
    initial_tokens = tokenizer.encode(text, add_special_tokens=False)
    total_tokens = len(initial_tokens)
    window_segments = []

    if total_tokens <= max_length:
        # Jika total token kurang dari max_length, tidak perlu sliding window
        return [tokenizer.encode(text, add_special_tokens=True)]

    for i in range(0, total_tokens, stride):
        # Memilih segmen token dengan mempertimbangkan token spesial
        window_segment = [tokenizer.cls_token_id] + initial_tokens[i:i + max_length - 2] + [tokenizer.sep_token_id]
        window_segments.append(window_segment)

    return window_segments

In [9]:
# Fungsi untuk menggabungkan token menjadi string
def tokens_to_string(tokens):
    return tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(tokens))

In [20]:
# Memuat dataset
df = pd.read_csv('Merged_data.csv')

In [21]:
# Pembersihan teks
df['cleaned_OCR_text'] = df['OCR_Text'].apply(clean_text)
# Sliding window tokenization
df['tokenized_segments'] = df['cleaned_OCR_text'].apply(sliding_window_tokenize)

In [22]:
# Padding dan Attention Mask
MAX_LEN = 512

def pad_segments(segments):
    return pad_sequences(segments, maxlen=MAX_LEN, padding='post', truncating='post')

def create_attention_masks(padded_segments):
    return np.where(padded_segments != 0, 1, 0)

In [23]:
df['padded_segments'] = df['tokenized_segments'].apply(pad_segments)
df['attention_masks'] = df['padded_segments'].apply(create_attention_masks)

In [16]:
# Memuat model mBERT
model = TFBertModel.from_pretrained('bert-base-multilingual-cased')

model.safetensors: 100%|██████████| 714M/714M [03:30<00:00, 3.39MB/s] 
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the check

In [27]:
# Fungsi untuk mengolah segmen dengan mBERT
def bert_encode(segments, attention_mask):
    input_ids = tf.constant(segments)
    attention_mask = tf.constant(attention_mask)

    last_hidden_states = model(input_ids, attention_mask=attention_mask)
    
    return last_hidden_states[0][:, 0, :].numpy()

In [28]:
# Mengolah setiap baris data dengan mBERT
df['bert_features'] = df.apply(lambda row: bert_encode(row['padded_segments'], row['attention_masks']), axis=1)
# Rata-ratakan vektor fitur
df['average_bert_features'] = df['bert_features'].apply(lambda x: np.mean(x, axis=0))

KeyboardInterrupt: 

In [ ]:
# Menyiapkan label
df['encoded_label'] = df['Label'].apply(lambda x: 1 if x == 'Fraud' else 0)

In [ ]:
df.to_csv('preprocessed.csv')